In [8]:
import warnings
import logging


warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [9]:
from utils import make_environments
from utils import pearl_utils
from configs  import defaults

from utils.reward_functions import log_reward_function,cumulative_reward_function,sharpe_reward_function
from utils. utils import make_hidden_dims
import optuna
from optuna.samplers import TPESampler
from neuralforecast.core import NeuralForecast
from Pearl.pearl.utils.instantiations.environments.gym_environment import GymEnvironment
from Pearl.pearl.utils.functional_utils.train_and_eval.online_learning import online_learning
import datetime

import numpy as np
import pickle
import boto3
import Keys

In [10]:
# s3=boto3.client('s3', aws_access_key_id=Keys.AWS_ACCESS_KEY, aws_secret_access_key=Keys.AWS_SECRET_KEY)
# agent_path=f'Agent/pearl_{defaults.model_name}_model.pkl'

# s3.download_file('coinbasetradehistory',f'pearl_{defaults.model_name}_model.pkl',agent_path)


In [11]:
from configs import defaults

In [12]:
# model=NeuralForecast.load('MultiHeadForecastingModel/')

In [13]:
reward_functions=[log_reward_function,cumulative_reward_function,sharpe_reward_function]
train_env,test_env=make_environments.make_envs(reward_function=log_reward_function)
study_name=f"{defaults.model_name}"
storage_name="sqlite:///PearlHPTuning.sqlite3"

test_env.observation_space.shape,train_env.action_space.n


Seed set to 12
Seed set to 5


['data_forex/oanda-USDJPY-1h.pkl']


100%|██████████| 78/78 [00:00<00:00, 12858.38it/s]
1it [00:00,  8.72it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

2024-11-05 11:00:00 2024-12-05 11:00:00


((24,), np.int64(2))

In [14]:
today=datetime.datetime.now().strftime('%Y-%m-%d')
study = optuna.load_study(study_name=study_name,
                            storage=storage_name,

                            )


In [27]:
# print(f"Best value: {study.best_value} (params: {study.best_params})")
best_trials=study.best_trials
best_trials=[best_trials[x] for x in [0,2,-2,-1]]
best_trials

[FrozenTrial(number=24, state=TrialState.COMPLETE, values=[1155.4950017267067, 60.4], datetime_start=datetime.datetime(2024, 12, 4, 21, 4, 19, 218752), datetime_complete=datetime.datetime(2024, 12, 4, 21, 10, 28, 132784), params={'reward_function': 0, 'algorithm': 'dqn', 'n_layers': 1, 'n_units': 64, 'training_rounds': 27, 'learning_rate': 2.4762447589798305e-05, 'discount_factor': 0.986225305621527, 'batch_size': 64, 'target_update_freq': 24, 'soft_update_tau': 0.5042013854773669, 'is_conservative': False, 'lstm': True, 'conservative_alpha': 0.7217810056026264, 'replay_buffer_size': 589, 'learn_after_episode': True, 'learning_steps': 18, 'n_epochs': 100}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'reward_function': CategoricalDistribution(choices=(0, 1, 2)), 'algorithm': CategoricalDistribution(choices=('dqn', 'ddqn')), 'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units': CategoricalDistribution(choices=(64, 128, 256, 512)), 'training_

In [19]:


agent_path=f'Agent/pearl_{defaults.model_name}_model.pkl'

agent_path


'Agent/pearl_USDJPY_model.pkl'

In [20]:
best_agents={}
for i in range(len(best_trials)):
    agent,learning_params=pearl_utils.load_agent_from_study(study_path=storage_name,
                                            study_name=study_name,
                                            action_space_dim=train_env.action_space.n,
                                            observation_space_dim=train_env.observation_space.shape[0],
                                            version=i)
    
    agent,profit,n_trades=pearl_utils.train_production_agent(agent,
                                learning_params,
                                train_env=train_env,
                                test_env=test_env,
                                save_path=agent_path)
    best_agents[profit]=agent
agent=best_agents[max(best_agents)]
    

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [03:43<?, ?it/s]
Profit: 1158.5073339320456, Number of Trades: 18.8: 100%|██████████| 100/100 [02:28<00:00,  1.48s/it]             


Testing Return AVG Profit: 1158.5073339320456, AVG Number of Trades: 18.8


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [02:37<?, ?it/s]
Profit: 820.4595693067306, Number of Trades: 19.6: 100%|██████████| 100/100 [02:08<00:00,  1.28s/it]             


Testing Return AVG Profit: 820.4595693067306, AVG Number of Trades: 19.6


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [02:40<?, ?it/s]


In [21]:
# agent,learning_params=pearl_utils.load_agent_from_study(study_path="sqlite:///pearl_hyper_parameters.sqlite3",
#                                             study_name='pearl-2024-11-12-hp-search',
#                                             action_space_dim=2,
#                                             observation_space_dim=30,
#                                             version=1)

# agent=pearl_utils.train_production_agent(agent,
#                              learning_params,
#                              train_env=train_env,
#                              test_env=train_env,
#                              save_path=agent_path)

In [22]:
agent=best_agents[max(best_agents)]


In [23]:
best_agents,max(best_agents)


({np.float64(1158.5073339320456): <Pearl.pearl.pearl_agent.PearlAgent at 0x3755ca620>,
  np.float64(1228.9217851381718): <Pearl.pearl.pearl_agent.PearlAgent at 0x375243b20>,
  np.float64(829.9526389650495): <Pearl.pearl.pearl_agent.PearlAgent at 0x30f9fa500>,
  np.float64(820.4595693067306): <Pearl.pearl.pearl_agent.PearlAgent at 0x37557e8c0>},
 np.float64(1228.9217851381718))

In [24]:
profit,n_trades=pearl_utils.test_pearl_model(agent,test_env)
print(f"Testing Return AVG Profit: {profit}, AVG Number of Trades: {n_trades}")

Profit: 1185.1559115659747, Number of Trades: 20.4: 100%|██████████| 100/100 [02:07<00:00,  1.27s/it]             

Testing Return AVG Profit: 1185.1559115659747, AVG Number of Trades: 20.4


In [25]:
from utils import save_utils


In [26]:
save_utils.save_agent(agent,agent_path)

Device: mps:0


In [ ]:
# pickle.dump(agent.policy_learner.state_dict(),open(agent_path,'wb'))

In [ ]:
# weights=pickle.load(open(agent_path,'rb'))

In [ ]:
# agent.policy_learner.load_state_dict(weights)

In [ ]:

# agent,learning_params=pearl_utils.load_agent_from_study(study_path="sqlite:///pearl_hyper_parameters.sqlite3",
#                                         study_name='pearl-2024-11-12-hp-search',
#                                         action_space_dim=2,
#                                         observation_space_dim=30)
# agent=load_agent_weights(agent,weight_path=agent_path)

In [ ]:
# profit,n_trades=pearl_utils.test_pearl_model(agent,test_env)
# print(f"Testing Return AVG Profit: {profit}, AVG Number of Trades: {n_trades}")